# IPPO Waste Analysis + ML (Colab Ready)

This notebook contains the **full workflow** in one place:
1. Data loading from Excel.
2. Preprocessing and cleaning.
3. Descriptive analysis and business KPIs.
4. Machine learning model training + comparison for waste prediction (`بالكيلو`).
5. Export of outputs for business use.


In [ ]:
# Colab setup (safe to run in Colab)
!pip -q install pandas numpy scikit-learn openpyxl joblib tabulate matplotlib seaborn


In [ ]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.multioutput import MultiOutputClassifier
from joblib import dump

pd.set_option("display.max_columns", 100)


## 1) Input file
- Upload your Excel file to Colab files panel, then set `DATASET_PATH`.
- Example file name from your project: `تقرير يومي عن انتاجيه  الجوده شهر يوليو_cleaned.xlsx`


In [ ]:
# Set your file path here
DATASET_PATH = "/content/تقرير يومي عن انتاجيه  الجوده شهر يوليو_cleaned.xlsx"
assert os.path.exists(DATASET_PATH), f"File not found: {DATASET_PATH}"


## 2) Preprocessing Functions


In [ ]:
EXPECTED_COLUMNS = [
    "بالكيلو", "الوزن", "فرز تاني", "محلي", "تصدير", "السمك", "المقاس",
    "الاوردر", "الفني", "الجوده", "خط الانتاج", "الورديه"
]

NUMERIC_COLUMNS = ["بالكيلو", "الوزن", "فرز تاني", "محلي", "تصدير", "السمك", "المقاس"]
TEXT_COLUMNS = ["الفني", "الجوده", "خط الانتاج", "الورديه", "الاوردر"]

LINE_NORMALIZATION = {
    "سليتر 1": "سليتر1",
    "سليتر 2": "سليتر2",
    "تناية 1": "تناية1",
    "تناية 2": "تناية2",
    "تناية 3": "تناية3",
    "تناية 4": "تناية4",
    "تناية1 ": "تناية1",
}


def normalize_whitespace(x):
    return " ".join(str(x).strip().split())


def normalize_text_columns(df, columns):
    for col in columns:
        if col in df.columns:
            df[col] = df[col].astype(str).map(normalize_whitespace)
            df[col] = df[col].replace({"nan": pd.NA, "": pd.NA})

    if "خط الانتاج" in df.columns:
        df["خط الانتاج"] = df["خط الانتاج"].replace(LINE_NORMALIZATION)

    return df


def load_and_preprocess_excel(path):
    raw = pd.read_excel(path, header=None)

    # Same structure handling as your original notebook
    work = raw.drop(columns=[0, 1, 9, 15], errors="ignore").copy()
    header_row = work.iloc[2].loc[1:]
    work.columns = header_row

    df = work.drop(index=work.index[0:27]).copy()
    available = [c for c in EXPECTED_COLUMNS if c in df.columns]
    df = df[available].copy()

    for col in NUMERIC_COLUMNS:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    must_have = [c for c in ["بالكيلو", "الفني", "خط الانتاج"] if c in df.columns]
    df = df.dropna(subset=must_have)

    df = normalize_text_columns(df, TEXT_COLUMNS)

    # Remove invalid/unreasonable values
    df = df[(df["بالكيلو"] >= 0) & (df["بالكيلو"] < 400)].copy()
    return df


def remove_outliers_iqr_per_line(df, value_col="بالكيلو"):
    if value_col not in df.columns or "خط الانتاج" not in df.columns:
        return df.copy()

    parts = []
    for _, g in df.groupby("خط الانتاج", dropna=False):
        if len(g) < 4:
            parts.append(g)
            continue
        q1 = g[value_col].quantile(0.25)
        q3 = g[value_col].quantile(0.75)
        iqr = q3 - q1
        lower = q1 - 1.5 * iqr
        upper = q3 + 1.5 * iqr
        parts.append(g[(g[value_col] >= lower) & (g[value_col] <= upper)])

    return pd.concat(parts, ignore_index=True)


In [ ]:
df_clean = load_and_preprocess_excel(DATASET_PATH)
df_model = remove_outliers_iqr_per_line(df_clean)

print("Rows after preprocessing:", len(df_clean))
print("Rows after per-line IQR filtering:", len(df_model))
df_model.head()


## 3) Descriptive Analysis + Business KPIs


In [ ]:
line_summary = (
    df_model.groupby("خط الانتاج")["بالكيلو"]
    .agg(mean="mean", median="median", std="std", count="count", min="min", max="max")
    .sort_values("mean", ascending=False)
)

operator_summary = (
    df_model.groupby("الفني")["بالكيلو"]
    .agg(avg_waste="mean", median="median", records="count")
    .sort_values(["avg_waste", "records"], ascending=[False, False])
)

line_operator_summary = (
    df_model.groupby(["خط الانتاج", "الفني"])["بالكيلو"]
    .agg(avg_waste="mean", records="count")
    .sort_values("avg_waste", ascending=False)
)

print("Top 10 high-waste lines")
display(line_summary.head(10))
print("Top 10 high-waste operators")
display(operator_summary.head(10))
print("Top 15 high-waste operator-line combos")
display(line_operator_summary.head(15))


In [ ]:
plt.figure(figsize=(12,5))
line_summary.head(10)["mean"].plot(kind="bar", color="#d95f02")
plt.title("Top Waste Lines by Mean Waste (kg)")
plt.ylabel("Mean Waste (kg)")
plt.xlabel("Production Line")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(12,5))
operator_summary.head(15)["avg_waste"].plot(kind="bar", color="#1b9e77")
plt.title("Top Operators by Average Waste (kg)")
plt.ylabel("Average Waste (kg)")
plt.xlabel("Operator")
plt.xticks(rotation=60)
plt.tight_layout()
plt.show()


## 4) Machine Learning Models to Predict `الجوده` and `الفني`

- This section trains multi-output classification models (two targets at once).
- Metrics report separate accuracy for each target plus an overall average.


In [ ]:
TARGETS = [c for c in ["الجوده", "الفني"] if c in df_model.columns]
if len(TARGETS) < 2:
    raise ValueError("Both targets 'الجوده' and 'الفني' must exist in the dataset.")

CATEGORICAL = [c for c in ["خط الانتاج", "الورديه", "الاوردر"] if c in df_model.columns]
NUMERIC = [c for c in ["بالكيلو", "الوزن", "فرز تاني", "محلي", "تصدير", "السمك", "المقاس"] if c in df_model.columns]
FEATURES = NUMERIC + CATEGORICAL

X = df_model[FEATURES].copy()
y = df_model[TARGETS].copy()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler())
        ]), NUMERIC),
        ("cat", Pipeline([
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("onehot", OneHotEncoder(handle_unknown="ignore"))
        ]), CATEGORICAL),
    ],
    remainder="drop"
)

base_models = {
    "LogisticRegression": LogisticRegression(max_iter=2000),
    "RandomForest": RandomForestClassifier(n_estimators=400, random_state=42, n_jobs=-1),
    "ExtraTrees": ExtraTreesClassifier(n_estimators=500, random_state=42, n_jobs=-1),
}

results = []
fitted = {}
cv = KFold(n_splits=5, shuffle=True, random_state=42)

for name, base_model in base_models.items():
    model = MultiOutputClassifier(base_model)
    pipe = Pipeline([
        ("prep", preprocessor),
        ("model", model)
    ])

    pipe.fit(X_train, y_train)
    pred = pipe.predict(X_test)

    acc_quality = accuracy_score(y_test[TARGETS[0]], pred[:, 0])
    acc_technician = accuracy_score(y_test[TARGETS[1]], pred[:, 1])
    acc_avg = (acc_quality + acc_technician) / 2

    cv_exact_match = cross_val_score(pipe, X_train, y_train, cv=cv, scoring="accuracy")

    results.append({
        "model": name,
        "Accuracy_الجوده": acc_quality,
        "Accuracy_الفني": acc_technician,
        "Accuracy_Mean": acc_avg,
        "CV_ExactMatch_Mean": cv_exact_match.mean(),
        "CV_ExactMatch_STD": cv_exact_match.std()
    })
    fitted[name] = pipe

metrics_df = pd.DataFrame(results).sort_values("Accuracy_Mean", ascending=False).reset_index(drop=True)
metrics_df


In [ ]:
best_model_name = metrics_df.iloc[0]["model"]
best_model = fitted[best_model_name]
print("Best model:", best_model_name)

# Feature importance (if supported by first estimator in MultiOutputClassifier)
model_obj = best_model.named_steps["model"]
if hasattr(model_obj, "estimators_") and hasattr(model_obj.estimators_[0], "feature_importances_"):
    feat_names = best_model.named_steps["prep"].get_feature_names_out()
    fi = pd.DataFrame({
        "feature": feat_names,
        "importance": model_obj.estimators_[0].feature_importances_
    }).sort_values("importance", ascending=False)
    display(fi.head(20))


## 5) Save outputs for business usage


In [ ]:
OUTPUT_DIR = "/content/outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

df_model.to_csv(f"{OUTPUT_DIR}/cleaned_data.csv", index=False)
line_summary.to_csv(f"{OUTPUT_DIR}/line_summary.csv")
operator_summary.to_csv(f"{OUTPUT_DIR}/operator_summary.csv")
line_operator_summary.to_csv(f"{OUTPUT_DIR}/line_operator_summary.csv")
metrics_df.to_csv(f"{OUTPUT_DIR}/model_metrics.csv", index=False)
dump(best_model, f"{OUTPUT_DIR}/best_quality_technician_model.joblib")

# Text report
with open(f"{OUTPUT_DIR}/MANAGEMENT_REPORT.txt", "w", encoding="utf-8") as f:
    f.write("IPPO Waste Analysis + ML Summary\n")
    f.write("================================\n\n")
    f.write(f"Rows used for modeling: {len(df_model)}\n")
    f.write("Targets: الجوده, الفني\n")
    f.write(f"Best model: {best_model_name}\n\n")
    f.write("Top waste lines:\n")
    f.write(line_summary.head(5).to_string())
    f.write("\n\nTop waste operators:\n")
    f.write(operator_summary.head(10).to_string())
    f.write("\n\nModel metrics:\n")
    f.write(metrics_df.to_string(index=False))

print("Saved outputs in:", OUTPUT_DIR)
print(sorted(os.listdir(OUTPUT_DIR)))


## 6) Business recommendations
- Use the saved model monthly to forecast expected `الجوده` assignment and responsible `الفني`.
- Monitor low-accuracy cases to identify unstable shifts, lines, or orders needing process standardization.
- Use target-specific accuracy (`Accuracy_الجوده`, `Accuracy_الفني`) to decide where to improve data quality.
- Track KPI trends together with model accuracy metrics every month.
